# Imports 

In [1]:
import sys
sys.path.append("../metaflow/")
import preprocess_fn

# Load the set from JSON

In [2]:
import json
import numpy as np

In [3]:
json_file = '../dataset/M20.json'

In [4]:
with open(json_file, 'r') as fp:
    raw_data = json.load(fp)
    cards = raw_data['cards']

n = len(cards)

# Dump the cards in a JSON

In [5]:
with open('../dataset/M20_cards.json', 'w') as fp:
    json.dump(cards, fp)

# Clean the data 

In [6]:
# import pyspark.sql.functions as fn
# from pyspark.sql.functions import udf
# from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType

In [7]:
# rules = {
#     "When CARDNAME enters the battlefield": "ETB_EFFECT",
    
#     "CARDNAME enters the battlefield tapped.": "ENTER_TAPPED",
    
#     "Defender (This creature can't attack.)": "DEFENDER",
#     "Defender": "DEFENDER",
    
#     "Flash (You may cast this spell any time you could cast an instant.)": "FLASH",
#     "Flash": "FLASH",
    
#     "Reach (This creature can block creatures with flying.)": "REACH",
#     "Reach": "REACH",
    
#     "Flying (This creature can't be blocked except by creatures with flying or reach.)": "FLYING",
#     "Flying": "FLYING",
    
#     "Haste (This creature can attack and {T} as soon as it comes under your control.)": "HASTE",
#     "Haste": "HASTE",
    
#     "Trample (This creature can deal excess combat damage to the player or planeswalker it's attacking.)": "TRAMPLE",
#     "Trample": "TRAMPLE",
    
#     "Vigilance (Attacking doesn't cause this creature to tap.)": "VIGILANCE",
#     "Vigilance": "VIGILANCE",

#     "Double strike (This creature deals both first-strike and regular combat damage.)": "DOUBLE_STRIKE",
#     "Double strike": "DOUBLE_STRIKE",

#     "Deathtouch (Any amount of damage this deals to a creature is enough to destroy it.)": "DEATHTOUCH",
#     "Deathtouch": "DEATHTOUCH",
    
#     "Protection from green (This creature can't be blocked, targeted, dealt damage, enchanted, or equipped by anything green.)": "PROTECTION_FROM_GREEN",
#     "Protection from red (This creature can't be blocked, targeted, dealt damage, enchanted, or equipped by anything red.)": "PROTECTION_FROM_RED",
#     "Protection from black (This creature can't be blocked, targeted, dealt damage, enchanted, or equipped by anything black.)": "PROTECTION_FROM_BLACK",
#     "Protection from blue (This creature can't be blocked, targeted, dealt damage, enchanted, or equipped by anything blue.)": "PROTECTION_FROM_BLUE",
#     "Protection from white (This creature can't be blocked, targeted, dealt damage, enchanted, or equipped by anything white.)": "PROTECTION_FROM_WHITE",
    
#     "(As this Saga enters and after your draw step, add a lore counter. Sacrifice after III.)": "SAGA_3",
#     "(As this Saga enters and after your draw step, add a lore counter. Sacrifice after IV.)": "SAGA_4"
# }

In [8]:
# @udf
# def udf_filter_text(name, text):    
#     if isinstance(text, str):
#         new_text = text
#         new_text = new_text.replace(name, 'CARDNAME')
#         for line in new_text:
#             for rule, replace in rules.items():
#                 new_text = new_text.replace(rule, replace)
                
#         print(new_text)

#         return new_text

In [9]:
df = spark.read.json('../dataset/M20_cards.json')

In [10]:
df.count()

345

## Filter out duplicate cards

In [11]:
df_filtered = preprocess_fn.remove_duplicate_cards(df)

In [12]:
num_cards = df_filtered.count()
print(f"Final number of cards {num_cards}")

Final number of cards 329


## Drop columns 

In [13]:
df_filtered = preprocess_fn.drop_columns(df_filtered)

## Filter the text

In [14]:
df_filtered = df_filtered.withColumn('filtered_text',
                                     preprocess_fn.udf_filter_text('name', 'text')
)

In [15]:
df_filtered.first()

Row(number='124', colorIdentity=['R'], colors=['R'], convertedManaCost=3.0, manaCost='{2}{R}', name='Act of Treason', power=None, rarity='common', subtypes=[], supertypes=[], text='Gain control of target creature until end of turn. Untap that creature. It gains haste until end of turn. (It can attack and {T} this turn.)', toughness=None, types=['Sorcery'], filtered_text='Gain control of target creature until end of turn. Untap that creature. It gains haste until end of turn. (It can attack and {T} this turn.)')

## Explode the selected arrays in a string, separated by "," 

In [16]:
df_filtered = preprocess_fn.explode_to_strs(df_filtered,
                                            ["colorIdentity", "types", "subtypes", "supertypes"]
)

## Encode newly created strings

### scikit-learn + pandas 

In [ ]:
# import pandas as pd
# import sklearn
# from sklearn import preprocessing

In [ ]:
# df_colorId = df_filtered.select("str_colorIdentity").distinct()

In [ ]:
# pd_allColorIds = df_filtered.select("str_colorIdentity").toPandas()
# pd_colorIds = df_colorId.toPandas()

In [ ]:
# le = preprocessing.LabelEncoder().fit(pd_colorIds)
# encoded_colorIds = le.transform(pd_allColorIds)

In [ ]:
# https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.StringIndexer

In [ ]:
# pd_colorIds = pd.DataFrame(encoded_colorIds, columns=['encoded_colorIdentity'])
# pd_colorIds['number'] = df_filtered.select("number").toPandas().astype('int32')

In [ ]:
# df_encoded_colorIds = spark.createDataFrame(pd_colorIds)

In [ ]:
# df_filtered = df_filtered.join(df_encoded_colorIds, on='number')

### spark + ml

In [18]:
# from pyspark.ml.feature import StringIndexer, IndexToString

In [19]:
# def encode_strings(df, cols):
#     for col in cols:
#         indexer = StringIndexer(inputCol=f"{col}", outputCol=f"encoded_{col}", stringOrderType='alphabetAsc')
#         model = indexer.fit(df)
#         df = model.transform(df)
        
#         indexer.save(f"/tmp/pyspark/stringindexer_{col}")
#         model.save(f"/tmp/pyspark/stringindexer_model_{col}")
#     return df

In [ ]:
df_filtered = preprocess_fn.encode_strings(df_filtered, ["rarity", "str_colorIdentity", "str_types", "str_subtypes", "str_supertypes"])

In [ ]:
df_filtered.select(["types", "str_types", "encoded_str_types"]).distinct().show()

In [ ]:
rm -rf "../models/pyspark/"

In [ ]:
mv "/tmp/pyspark/" "../models/"

In [ ]:
# df_filtered.select(["str_colorIdentity", "encoded_colorIdentity"]).limit(5).show()

In [ ]:
# df_filtered.select(["str_colorIdentity", "encoded_colorIdentity"]).distinct().show()

## Count the number of colors

In [ ]:
df_filtered = df_filtered.withColumn("num_colors", fn.size("colors"))

#  Create an SQL table

In [ ]:
df_filtered.createOrReplaceTempView("cards")

In [ ]:
tbl = spark.sql("""
    SELECT
        CAST(number as Integer), 
        rarity,
        name,
        CAST(convertedManaCost as Integer),
        CAST(num_colors as Integer) as numColors,
        str_colorIdentity as colorIdentity,
        CAST(encoded_str_colorIdentity as Integer) as encodedColorIdentity,
        str_types as types,
        CAST(encoded_str_types as Integer) as encodedTypes,
        str_subtypes as subTypes,
        CAST(encoded_str_subtypes as Integer) as encodedSubTypes,
        str_supertypes as superTypes,
        CAST(encoded_str_supertypes as Integer) as encodedSuperTypes,
        text as originalText,
        filtered_text as filteredText,
        CAST(power as Integer),
        CAST(toughness as Integer)
    FROM
        cards
""")

# Save to Parquet

In [ ]:
tbl.write.mode("overwrite").parquet('/tmp/M20_cards.parquet')

In [ ]:
## this fails
# tbl.write.mode("overwrite").parquet('dataset/THB_cards.parquet')

In [ ]:
rm -rf "../dataset/M20_cards.parquet/"

In [ ]:
mv "/tmp/M20_cards.parquet" "../dataset/"